# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

$Gini(A)=\displaystyle\sum_{j=1}^{2}\cfrac{|D_j|}{|D|}Gini(D_i)\;\;,\;\;where\;\;\;Gini(D_i)=1-\displaystyle\sum_{j=1}^{x}P_j^2$

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    length = len(df[label])             # data의 개수 입니다.
    x = len(df[label].unique())         # 위의 수식에서 오른쪽의 x값은 구하고자 하는 set D의 label의 개수입니다.
    y = 0                               # 누적 합 할 변수를 지정해 줍니다.
    for i in range(x):                  # 총 unique label의 수만큼 iterate합니다.
        cnt = df[label].value_counts()  # 해당 set의 unique value별 count를 계산합니다.
        p = cnt[i]/length               # P를 직접 계산합니다. = (counts of unique values)/(전체 데이터 수)
        y += p*p                        # y에 제곱한 값은 누적합 합니다.
    gini = 1 - y                        # 최종적으로 gini index를 계산 합니다. 
    return gini

In [4]:
get_gini(pd_data,'income')

0.653061224489796

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
# 처음에 올려주신 결과를 보지 못해서 계산 값은 다 같으나 중간 과정이 살짝 다릅니다.ㅠㅠ


from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 ( unique values )
    result=[]
    mid = []
    for i in range( len(uniques)//2 ):     # combination을 이용할 것입니다. 따라서 모든 개수만큼 iterate할 필요가 없습니다.
                                           # 예를들어 unique valu가 6개라면 (5,1) (4,2) (3,3)이렇게 됩니다. 
                                           # 이때 실제 combination계산은 5,4,3만 합니다.
                                           # 그러면 나머지 pair는 전체에서 빼주면 되니 자동적으로 완성 됩니다.
                                           # 따라서 (unique value 개수) // 2 만큼만 iterate합니다.
                                           
        half = combinations(uniques, len(uniques) - i - 1)   # unique value가 6개 라면 5, 4, 3 만을 계산 합니다.
        mid += (list(half))                                  # 계산된 combination을 list의 형태로 중간 저장합니다.
        
        for j in mid:                                        # unique value가 6개 라면 (5, 4, 3)으로 3번의 iteration을 진행합니다.
            rest = uniques.copy()                            # combination으로 계산 되지 않은 pair가 계산 됩니다.
            rest = [x for x in rest if x not in list(j)]     # 즉, unique에서 combination으로 계산 된 것을 빼줍니다.
            result.append([list(j), rest])                   # save
    
    if len(uniques) == 2:                  # unique value 개수가 2개일때는 추가적인 작업이 필요 합니다.
        result = [result[0]]               # 같은 list가 두번 반복되므로 하나만을 선택 합니다.
                                           # 또한 선택만 하게 되면 dim이 달라져 하나 추가해 줍니다.
    
    return result           # 최종 결과는 전체 list안에 하나의 pair list가 있는 형태로 반환 됩니다.

In [6]:
get_binary_split(pd_data,'age')

[[['youth', 'middle_aged'], ['senior']],
 [['youth', 'senior'], ['middle_aged']],
 [['middle_aged', 'senior'], ['youth']]]

In [7]:
df_test =  pd.DataFrame({"income" : ['A', 'B']}) 
get_binary_split(df_test,'income')

[[['A'], ['B']]]

In [8]:
df_test =  pd.DataFrame({"income" : ['A', 'B', 'C']}) 
get_binary_split(df_test,'income')

[[['A', 'B'], ['C']], [['A', 'C'], ['B']], [['B', 'C'], ['A']]]

In [9]:
df_test =  pd.DataFrame({"income" : ['A', 'B', 'C', 'D']}) 
get_binary_split(df_test,'income')

[[['A', 'B', 'C'], ['D']],
 [['A', 'B', 'D'], ['C']],
 [['A', 'C', 'D'], ['B']],
 [['B', 'C', 'D'], ['A']],
 [['A', 'B', 'C'], ['D']],
 [['A', 'B', 'D'], ['C']],
 [['A', 'C', 'D'], ['B']],
 [['B', 'C', 'D'], ['A']],
 [['A', 'B'], ['C', 'D']],
 [['A', 'C'], ['B', 'D']],
 [['A', 'D'], ['B', 'C']],
 [['B', 'C'], ['A', 'D']],
 [['B', 'D'], ['A', 'C']],
 [['C', 'D'], ['A', 'B']]]

In [10]:
df_test =  pd.DataFrame({"income" : ['A', 'B', 'C', 'D', 'E']}) 
get_binary_split(df_test,'income')

[[['A', 'B', 'C', 'D'], ['E']],
 [['A', 'B', 'C', 'E'], ['D']],
 [['A', 'B', 'D', 'E'], ['C']],
 [['A', 'C', 'D', 'E'], ['B']],
 [['B', 'C', 'D', 'E'], ['A']],
 [['A', 'B', 'C', 'D'], ['E']],
 [['A', 'B', 'C', 'E'], ['D']],
 [['A', 'B', 'D', 'E'], ['C']],
 [['A', 'C', 'D', 'E'], ['B']],
 [['B', 'C', 'D', 'E'], ['A']],
 [['A', 'B', 'C'], ['D', 'E']],
 [['A', 'B', 'D'], ['C', 'E']],
 [['A', 'B', 'E'], ['C', 'D']],
 [['A', 'C', 'D'], ['B', 'E']],
 [['A', 'C', 'E'], ['B', 'D']],
 [['A', 'D', 'E'], ['B', 'C']],
 [['B', 'C', 'D'], ['A', 'E']],
 [['B', 'C', 'E'], ['A', 'D']],
 [['B', 'D', 'E'], ['A', 'C']],
 [['C', 'D', 'E'], ['A', 'B']]]

In [11]:
df_test =  pd.DataFrame({"income" : ['A', 'B', 'C', 'D', 'E', 'F']}) 
get_binary_split(df_test,'income')

[[['A', 'B', 'C', 'D', 'E'], ['F']],
 [['A', 'B', 'C', 'D', 'F'], ['E']],
 [['A', 'B', 'C', 'E', 'F'], ['D']],
 [['A', 'B', 'D', 'E', 'F'], ['C']],
 [['A', 'C', 'D', 'E', 'F'], ['B']],
 [['B', 'C', 'D', 'E', 'F'], ['A']],
 [['A', 'B', 'C', 'D', 'E'], ['F']],
 [['A', 'B', 'C', 'D', 'F'], ['E']],
 [['A', 'B', 'C', 'E', 'F'], ['D']],
 [['A', 'B', 'D', 'E', 'F'], ['C']],
 [['A', 'C', 'D', 'E', 'F'], ['B']],
 [['B', 'C', 'D', 'E', 'F'], ['A']],
 [['A', 'B', 'C', 'D'], ['E', 'F']],
 [['A', 'B', 'C', 'E'], ['D', 'F']],
 [['A', 'B', 'C', 'F'], ['D', 'E']],
 [['A', 'B', 'D', 'E'], ['C', 'F']],
 [['A', 'B', 'D', 'F'], ['C', 'E']],
 [['A', 'B', 'E', 'F'], ['C', 'D']],
 [['A', 'C', 'D', 'E'], ['B', 'F']],
 [['A', 'C', 'D', 'F'], ['B', 'E']],
 [['A', 'C', 'E', 'F'], ['B', 'D']],
 [['A', 'D', 'E', 'F'], ['B', 'C']],
 [['B', 'C', 'D', 'E'], ['A', 'F']],
 [['B', 'C', 'D', 'F'], ['A', 'E']],
 [['B', 'C', 'E', 'F'], ['A', 'D']],
 [['B', 'D', 'E', 'F'], ['A', 'C']],
 [['C', 'D', 'E', 'F'], ['A', 'B']],
 

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.  
$Gini(A)=\displaystyle\sum_{j=1}^{2}\cfrac{|D_j|}{|D|}Gini(D_i)\;\;,\;\;where\;\;\;Gini(D_i)=1-\displaystyle\sum_{j=1}^{x}P_j^2$

In [12]:
def get_attribute_gini_index(df, attribute, label):
    binary_split = get_binary_split(df, attribute)
    result = {}
    length = len(df)    # 전체의 길이 입니다 위의 식에서 |D|를 의미합니다.

    for i in binary_split:   # 전체 binary_split을 iterate합니다.
                                                            # 아예 새로운 dataframe을 만들어 줍니다.
        g1_df = pd.DataFrame(columns = df.columns.tolist()) # 위의 식에서 왼쪽식의 j가 1일때 입니다. 빈 dataframe에 추가합니다.
        g2_df = df.copy()                                   # 위의 식에서 왼쪽식의 j가 2일때 입니다. 원래 dataframe에서 제거 합니다.
                                                            
        for j in i[0]:                                      # binary_split 함수에서도 사용한 개념인데 pair이기 때문에 하나만 계산하면 충분합니다.
                                                            # 예를들어 'age'이면 i[0]는 ['youth', 'middle_aged']가 되고
                                                            # j는 각 원소를 iterate합니다.
            g1_df = g1_df.append( df[df[attribute] == j] )  # 각 원소가 있으면 추가 합니다.
            g2_df.drop(g2_df[g2_df[attribute] == j].index, inplace=True) # 같은 방식으로 각 원소가 있으면 삭제 합니다.
        g1 = get_gini(g1_df, label)    # 이는 위에서 오른쪽의 계산 과정입니다.
        g2 = get_gini(g2_df, label)    # 처리된 dataframe을 가지고 gini를 계산합니다.
        d1 = len(g1_df)                # 위의 식에서 |D_1|입니다.
        d2 = len(g2_df)                # 위의 식에서 |D_2|입니다.
        value = (d1 / length) * g1 + (d2 / length) * g2 
        
        result[str(i)] = value         # save
    
    return result

In [13]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{"[['youth', 'middle_aged'], ['senior']]": 0.45714285714285713,
 "[['youth', 'senior'], ['middle_aged']]": 0.35714285714285715,
 "[['middle_aged', 'senior'], ['youth']]": 0.3936507936507937}

In [14]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{"[['high', 'medium'], ['low']]": 0.45,
 "[['high', 'low'], ['medium']]": 0.4583333333333333,
 "[['medium', 'low'], ['high']]": 0.4428571428571429}

In [15]:
get_attribute_gini_index(pd_data, 'student', 'class_buys_computer')

{"[['no'], ['yes']]": 0.3673469387755103}

In [16]:
get_attribute_gini_index(pd_data, 'credit_rating', 'class_buys_computer')

{"[['fair'], ['excellent']]": 0.42857142857142855}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [17]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print("Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(pd_data, feature, label).values()), 4))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [18]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{"[['youth', 'middle_aged'], ['senior']]": 0.45714285714285713,
 "[['youth', 'senior'], ['middle_aged']]": 0.35714285714285715,
 "[['middle_aged', 'senior'], ['youth']]": 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [19]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']
df_2 = pd_data[pd_data['age'] != 'middle_aged']

In [20]:
df_1.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [21]:
df_2.head() #split 결과 확인

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [22]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

rest_features = ["income", "student", "credit_rating"]     # 'age'를 제외한 변수들 입니다.

for feature in rest_features:
    print("Age : Middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df_1, feature, label).values()), 4))

print("#############################################################")

for feature in rest_features:
    print("Age : Middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df_2, feature, label).values()), 4))

Age : Middle_aged, Minimum Gini index of income : 0.0
Age : Middle_aged, Minimum Gini index of student : 0.0
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0
#############################################################
Age : Middle_aged, Minimum Gini index of income : 0.375
Age : Middle_aged, Minimum Gini index of student : 0.32
Age : Middle_aged, Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "450">

In [23]:
from math import log2

def getEntropy(df, feature) :
    cnt = df[feature].value_counts()   # unique별로 count합니다.
    legth = len(df)                    # 전체 개수 입니다.
    entropy = 0
    
    for i in cnt:                      # # unique별 count를 iterate합니다.
        p = i / legth
        entropy -= p*log2(p)
    
   
    
    return(entropy)

In [24]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

$Gain(S,A)=E(S)-I(S,A)$  
$where, \;\; I(S,A)=\displaystyle\sum_{i} \cfrac{|S_i|}{|S|}E(S_i)$ 이고 $i$는 $A$ class의 unique 개수 입니다.  

In [25]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
    info_D = getEntropy(df, feature)    # 목표변수에 대한 Entropy 구하기   **위의 식에서 보면 E(S)입니다.
    columns = list(df.loc[:, df.columns != feature])     # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    
    
    for i in columns:     # column별로 iterate합니다.
        I = 0             # 위의 식에서 I(S,A)입니다.
        i_list = list(df[i].unique())   # 위의 식에서 i입니다.
        for j in i_list:
            len_S = len(df[feature])     # 위의 식에서 |S|입니다.
            len_Si = sum(df[i] == j)     # 위의 식에서 |S_i|입니다.
            I += (len_Si / len_S)*getEntropy(df[df[i] == j], feature)   # |S_i|/|S| * E(S_i)입니다.
        result[i] = info_D - I   # 위의 식에서 E(S) - I(S,A) 입니다.
        
    return result

In [26]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}